# IBM Data Science Capstone Course 

## Week3 Part 1
- Build a dataframe of the postal code of each neighborhood

## Week3 Part 2
- Toronto geolocation coordinates

## Week3 Part 3
- Explore and cluster the neighborhoods in Toronto

### Week3 Part 1
- Build a dataframe of the postal code of each neighborhood

#### 1. Import libraries

In [5]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from bs4 import BeautifulSoup # library to parse HTML and XML documents

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#### 2. Scrape  data from Wikipedia page

In [3]:
#send a GET request to the web page
data_f=requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

In [7]:
# parse data from the html into a beautifulsoup object
soup_f = BeautifulSoup(data_f, 'html.parser')

In [8]:
#three lists to store table data
postalCodeList = []
boroughList = []
neighborhoodList = []

#### Using BeautifulSoup

In [10]:
# find the table
soup_f.find('table').find_all('tr')

# find all the rows of the table
soup_f.find('table').find_all('tr')

# for each row of the table, find all the table data
for row in soup_f.find('table').find_all('tr'):
    cells = row.find_all('td')

In [13]:
# append the data into the respective lists
for row in soup_f.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\n')) 
        # must avoid new lines in neighborhood cell

In [14]:
#a new DataFrame from the three lists
toronto_df = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})

toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### 3. "Not assigned" is removed from the table

In [16]:
toronto_df_dropNa=toronto_df[toronto_df.Borough !="Not assigned"].reset_index(drop=True)

In [19]:
toronto_df_dropNa.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


#### 4. Group neighborhoods 

In [21]:
# grouping neighborhoods in the same borough
toronto_df_grouped = toronto_df_dropNa.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))

In [22]:
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern, Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union, Highla..."
2,M1E,Scarborough,"Guildwood, Morningside, West Hill, Guildwood, ..."
3,M1G,Scarborough,"Woburn, Woburn"
4,M1H,Scarborough,"Cedarbrae, Cedarbrae"


#### 5. For Neighborhood="Not assigned", make the value the same as Borough

In [23]:
# for Neighborhood="Not assigned", make the value the same as Borough
for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]

In [24]:
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern, Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union, Highla..."
2,M1E,Scarborough,"Guildwood, Morningside, West Hill, Guildwood, ..."
3,M1G,Scarborough,"Woburn, Woburn"
4,M1H,Scarborough,"Cedarbrae, Cedarbrae"


#### 6. Create a new test data frame

In [25]:
# create a new test dataframe
column_names = ["PostalCode", "Borough", "Neighborhood"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_grouped[toronto_df_grouped["PostalCode"]==postcode], ignore_index=True)

In [26]:
test_df

,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,"Central Bay Street, Central Bay Street"
1,M2H,North York,"Hillcrest Village, Hillcrest Village"
2,M4B,East York,"Woodbine Gardens, Parkview Hill, Woodbine Gard..."
3,M1J,Scarborough,"Scarborough Village, Scarborough Village"
4,M4G,East York,"Leaside, Leaside"
5,M4M,East Toronto,"Studio District, Studio District"
6,M1R,Scarborough,"Maryvale, Wexford, Maryvale, Wexford"
7,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."
8,M9L,North York,"Humber Summit, Humber Summit"
9,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo..."


#### 7. The number of rows

In [29]:
toronto_df_grouped.shape

(103, 3)

### Week3 Part 2
- Toronto geolocation coordinates

#### 1. Load the coordinatas form csv file

In [33]:
##loading the coordinates and rename the column "PostalCode"
coordinates_pd=pd.read_csv("Geospatial_Coordinates.csv")
coordinates_pd.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
coordinates_pd.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### 2. Merge the two tables into one

In [35]:
# merge two table on the column "PostalCode"
toronto_df_new = toronto_df_grouped.merge(coordinates_pd, on="PostalCode", how="left")
toronto_df_new.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern, Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union, Highla...",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill, Guildwood, ...",43.763573,-79.188711
3,M1G,Scarborough,"Woburn, Woburn",43.770992,-79.216917
4,M1H,Scarborough,"Cedarbrae, Cedarbrae",43.773136,-79.239476


#### 3. Results

In [41]:
# create a new test dataframe
column_names = ["PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_new[toronto_df_new["PostalCode"]==postcode], ignore_index=True)
    


In [42]:
test_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,"Central Bay Street, Central Bay Street",43.657952,-79.387383
1,M2H,North York,"Hillcrest Village, Hillcrest Village",43.803762,-79.363452
2,M4B,East York,"Woodbine Gardens, Parkview Hill, Woodbine Gard...",43.706397,-79.309937
3,M1J,Scarborough,"Scarborough Village, Scarborough Village",43.744734,-79.239476
4,M4G,East York,"Leaside, Leaside",43.709060,-79.363452
5,M4M,East Toronto,"Studio District, Studio District",43.659526,-79.340923
6,M1R,Scarborough,"Maryvale, Wexford, Maryvale, Wexford",43.750072,-79.295849
7,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437
8,M9L,North York,"Humber Summit, Humber Summit",43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420


### Week3 Part 3
- Explore and cluster the neighborhoods in Toronto

#### 1.  To get the latitude and longitude values of Toronto "Geopy library" is using

In [43]:
address = 'Toronto'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### 2. Create a map of Toronto

In [44]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)

In [45]:
map_toronto

#### 3. Filter for "Toronto" words

In [46]:
# filter borough names that contain the word "Toronto"
borough_names = list(toronto_df_new.Borough.unique())

borough_with_toronto = []

for x in borough_names:
    if "toronto" in x.lower():
        borough_with_toronto.append(x)

In [47]:
borough_with_toronto

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']

In [48]:
#create a new DataFrame with only boroughs that contain the word Toronto
toronto_df_new = toronto_df_new[toronto_df_new['Borough'].isin(borough_with_toronto)].reset_index(drop=True)
print(toronto_df_new.shape)

(38, 5)


In [49]:
toronto_df_new.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,"The Beaches, The Beaches",43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale, The Danforth Wes...",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar, The Beaches We...",43.668999,-79.315572
3,M4M,East Toronto,"Studio District, Studio District",43.659526,-79.340923
4,M4N,Central Toronto,"Lawrence Park, Lawrence Park",43.728020,-79.388790


In [50]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  

In [51]:
map_toronto

#### 4. Clusterning
